### Exercise 1

In [4]:
import cobra
from cobra import Model, Reaction, Metabolite
model = cobra.io.load_json_model('e_coli_core.json')
model

import csv
activities = {}
with open('e_coli_core_expression.csv', mode='r', newline='') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        activities[row[0].strip()] = float(row[1].strip())

1/a: Upon inspecting the Escher map, we can see that contrary to flux, maximum activity is not equal along a linear path. This makes sense since the maximum activity is dependent on the individual capacities of each enzyme. These are inherent characteristics of the enzymes and are not constrained by mass balance.

1/b: Let's look at 2 such reactions.

In [5]:
model.reactions.get_by_id("EX_glc__D_e")

Reaction identifier,EX_glc__D_e
Name,D-Glucose exchange
Memory address,0x30b8e3410
Stoichiometry,glc__D_e <=> D-Glucose <=>
GPR,
Lower bound,-10.0
Upper bound,1000.0


In [6]:
model.reactions.get_by_id("BIOMASS_Ecoli_core_w_GAM")

Reaction identifier,BIOMASS_Ecoli_core_w_GAM
Name,Biomass Objective Function with GAM
Memory address,0x30b840cb0
Stoichiometry,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c... 1.496 3-Phospho-D-glycerate + 3.7478 Acetyl-CoA + 59.81 ATP C10H12N5O13P3 + 0.361 D-Erythrose 4-phosphate + 0.0709 D-Fructose 6-phosphate + 0.129 Glyceraldehyde 3-phosphate + 0.205 D-Glucose...
GPR,
Lower bound,0.0
Upper bound,1000.0


The first reaction is an exchange reaction, which moves metabolites but is not catalyzed by an enzyme, hence no genes to link.
The second is a biomass reaction, which is a conceptual reaction representing the growth rate of the organism, again no gene to link since it is an artificial representation.
In both cases the reaction is not catalyzed by an enzyme, thus there is no associated GPR. We cannot assign a maximal activity to either of these because gene expression data is mapped to reactions using GPR.

### Exercise 2

In [7]:
for reaction in model.reactions:
    if reaction.id in activities:
        value = activities[reaction.id]
        #setting the upper and lower flux for reversible ones
        if reaction.reversibility:
            reaction.lower_bound = -value
            reaction.upper_bound = value
        else:
            #only the upper for non reverrsible ones
            reaction.upper_bound = value
    #we use the high absolute default bounds for glucose
    if reaction.id == 'EX_glc__D_e':
        reaction.lower_bound = -1000.0
        reaction.upper_bound = 1000.0
        #we do nothing for ATPM
    elif reaction.id == 'ATPM':
        pass

for reaction in model.reactions:
    print(reaction.id, reaction.lower_bound, reaction.upper_bound)

PFK 0.0 12.12
PFL 0.0 1.0
PGI -13.12 13.12
PGK -23.13 23.13
PGL 0.0 8.12
ACALD -1.16 1.16
AKGt2r -3.1 3.1
PGM -20.01 20.01
PIt2r -6.03 6.03
ALCD2x -9.01 9.01
ACALDt -2.29 2.29
ACKr -1.19 1.19
PPC 0.0 2.56
ACONTa -25.35 25.35
ACONTb -25.35 25.35
ATPM 8.39 1000.0
PPCK 0.0 25.23
ACt2r -3.23 3.23
PPS 0.0 2.5
ADK1 -30.57 30.57
AKGDH 0.0 24.35
ATPS4r -60.5 60.5
PTAr -4.47 4.47
PYK 0.0 26.78
BIOMASS_Ecoli_core_w_GAM 0.0 1000.0
PYRt2 -1.26 1.26
CO2t -30.45 30.45
RPE -5.67 5.67
CS 0.0 20.56
RPI -5.56 5.56
SUCCt2_2 0.0 2.36
CYTBD 0.0 40.56
D_LACt2 -4.56 4.56
ENO -28.78 28.78
SUCCt3 0.0 6.67
ETOHt2r -2.34 2.34
SUCDi 0.0 24.34
SUCOAS -20.6 20.6
TALA -4.45 4.45
THD2 0.0 4.5
TKT1 -3.34 3.34
TKT2 -3.34 3.34
TPI -45.56 45.56
EX_ac_e 0.0 1000.0
EX_acald_e 0.0 1000.0
EX_akg_e 0.0 1000.0
EX_co2_e -1000.0 1000.0
EX_etoh_e 0.0 1000.0
EX_for_e 0.0 1000.0
EX_fru_e 0.0 1000.0
EX_fum_e 0.0 1000.0
EX_glc__D_e -1000.0 1000.0
EX_gln__L_e 0.0 1000.0
EX_glu__L_e 0.0 1000.0
EX_h_e -1000.0 1000.0
EX_h2o_e -1000.0 100

### Exercise 3

a) FVA for all reactions (how much up or down does flux allow to go while rest of reactions stays working)

In [149]:
from cobra.flux_analysis import flux_variability_analysis as FVA

#FVA
fva_df = FVA(model, model.reactions, 0.0, False).reset_index().rename(columns={"index":"rxn_id"})
print(f"FVA reactions: {len(fva_df)}")
fva_df.to_csv("ex3_fva_all_reactions.csv", index=False)
fva_df


FVA reactions: 95


,rxn_id,minimum,maximum
0,PFK,0.334677,11.153556
1,PFL,0.000000,1.000000
2,PGI,-0.085323,9.020351
3,PGK,-17.596002,-0.879355
4,PGL,0.000000,4.522388
...,...,...,...
90,NADH16,1.831111,20.260000
91,NADTRHD,0.000000,1.260000
92,NH4t,0.000000,3.450000
93,O2t,0.500000,12.256000


b) reactions that have less flux than they could potentially handle 

In [152]:
#for small values
tol = 1e-9



#expression table
expr = pd.read_csv("e_coli_core_expression.csv")
expr.columns = [c.strip().lower() for c in expr.columns]
id_col  = [c for c in expr.columns if c in ("rxn_id","reaction","reaction_id","# reaction id","id")][0]
val_col = [c for c in expr.columns if ("activity" in c) or c in ("value","max","max_activity","maximal_activity")][0]
expr = expr[[id_col, val_col]].rename(columns={id_col: "rxn_id", val_col: "max_activity"})
expr["rxn_id"] = expr["rxn_id"].astype(str).str.strip()
expr = expr.dropna(subset=["rxn_id"])
expr_set = set(expr["rxn_id"]) & {r.id for r in model.reactions}



# we aint using FORt
exclude = {"FORt"} & {r.id for r in model.reactions}
#upper bounds
ub_map = {r.id: float(r.upper_bound) for r in model.reactions}

rows = []
for _, row in fva_df.iterrows():
    rid = row["rxn_id"]
    if rid in exclude or rid not in expr_set:
        continue
    fmax = float(row["maximum"])
    forward_max = max(0.0, fmax)    
    ub = ub_map[rid]
    if forward_max > tol and forward_max < ub - tol:
        rows.append([rid, float(row["minimum"]), float(row["maximum"]), ub])


partb_df = pd.DataFrame(rows, columns=["rxn_id","fva_min","fva_max","expr_ub"]).sort_values("rxn_id")
partb_df.to_csv("ex3_partb_expr_limited_forward_range.csv", index=False)
print(f"3b count: {len(partb_df)}")
print(partb_df.to_string(index=False))

3b count: 36
 rxn_id       fva_min   fva_max  expr_ub
 ACONTa -2.782184e-17  5.775556    25.35
 ACONTb  0.000000e+00  5.775556    25.35
   ADK1  0.000000e+00  2.500000    30.57
  AKGDH  0.000000e+00  4.126039    24.35
 ATPS4r  4.266667e-01 27.641000    60.50
   CO2t -1.367477e+01  1.615333    30.45
     CS  0.000000e+00  5.775556    20.56
  CYTBD  1.000000e+00 24.512000    40.56
    ENO  8.793548e-01 17.038932    28.78
    FBA  3.346774e-01  8.803556    24.45
    FUM -2.560000e+00  4.252000    24.35
G6PDH2r  0.000000e+00  4.522388     5.68
   GAPD  8.793548e-01 17.596002    26.13
 GLCpts  4.794286e-01  9.493300    16.34
  GLUDy -3.450000e+00  3.450000     7.35
   GLUN  0.000000e+00  3.450000     6.25
  GLUSy  0.000000e+00  3.450000     7.35
    GND  0.000000e+00  4.522388     6.72
 ICDHyr  3.987053e-16  5.202131    15.06
    ICL  0.000000e+00  4.252000    14.07
   MALS  1.196116e-15  4.252000     9.12
    O2t  5.000000e-01 12.256000    40.67
    PDH  0.000000e+00 10.931107    32.32
   

There are 36 of these restricted flux reactions. They cannot reach this because of some other constraints like > substrate uptake limits, ATP balance, competition at pathway branch points

c) reactions which have a positive minimum FVA (so always running(

In [150]:
#for small values
tol = 1e-9

#if it has a positive minimum 
partc_df = (fva_df.loc[fva_df["minimum"] > tol, ["rxn_id","minimum","maximum"]]
            .rename(columns={"minimum":"fva_min","maximum":"fva_max"})
            .sort_values("rxn_id"))
print("3c count:", len(partc_df))

partc_df.to_csv("ex3_partc_positive_minimum_flux.csv", index=False)
partc_df

3c count: 12


,rxn_id,fva_min,fva_max
15,ATPM,8.390000,39.983750
21,ATPS4r,0.426667,27.641000
31,CYTBD,1.000000,24.512000
33,ENO,0.879355,17.038932
55,EX_h2o_e,0.538000,18.590873
63,FBA,0.334677,8.803556
72,GAPD,0.879355,17.596002
73,GLCpts,0.479429,9.493300
90,NADH16,1.831111,20.260000
93,O2t,0.500000,12.256000


so there are a total of 12 reactions with a positive minimal flux. This is because they do things like transporting important things, respiratory or energy actions, ATP, and so on which are crucial for the cell no matter the state 

### Exercise 4

**4.a)** Carry out an FBA optimization of biomass and report the maximal biomass production rate in the presence of the implemented constraints.

In [8]:
#optimize the model for biomass production
solution = model.optimize()

#print the maximal biomass production rate
print("Maximal biomass production rate:")
print(solution.objective_value)

Maximal biomass production rate:
0.4847030979705454


**4.b)** Determine the “bottleneck(s)” – i. e. those reaction(s) whose flux reaches its maximum. 

In [10]:
print("Identifying bottlenecks (reactions at maximum flux):")
#a small tolerance is used for comparing floating point numbers
tolerance = 1e-6 

for reaction in model.reactions:
    #check if the reaction has a flux value in the solution
    if reaction.id in solution.fluxes:
        flux = solution.fluxes[reaction.id]
        upper_bound = reaction.upper_bound
        
        #check if the flux is very close to the upper bound
        if flux > 0 and abs(flux - upper_bound) < tolerance:
            print(f"- {reaction.id} is a bottleneck. Flux: {flux:.4f}, Bound: {upper_bound:.4f}")

Identifying bottlenecks (reactions at maximum flux):
- PFL is a bottleneck. Flux: 1.0000, Bound: 1.0000
- THD2 is a bottleneck. Flux: 4.5000, Bound: 4.5000
- NADH16 is a bottleneck. Flux: 20.2600, Bound: 20.2600


**4.c)** The model does not use (have non-zero flux for) all reactions in the model constrained with maximal reaction activities. Pick a reaction with maximal reaction activity constraints and zero flux in the optimal solution, and explain from the network context why it carries not carry flux. (Hint: Use the online demo again.) 

In [13]:
print("Finding a constrained but unused reaction:")
for reaction_id, max_activity in activities.items():
    #check if the reaction exists in the model and the solution
    if reaction_id in solution.fluxes:
        #check if the flux is essentially zero
        if abs(solution.fluxes[reaction_id]) < tolerance:
            print(f"Candidate: {reaction_id} has zero flux.")
            #we only need to find one
            break

Finding a constrained but unused reaction:
Candidate: AKGt2r has zero flux.


The analysis identified the reaction `AKGt2r` as a reaction that has a maximal activity constraint from the expression data but carries zero flux in the biomass-optimized solution. The reaction is defined as **akg_e + h_e ⇌ akg_c + h_c**. Judging by the network on the online demo, either `akg_e` or `h_e` must be missing for the reaction to take place, that could be if we only feed glucose to the cell for example.